In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os, warnings, tiktoken
from datetime import datetime, timezone
from keys import config

from langchain.llms import Ollama
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

warnings.filterwarnings("ignore")

# **Load dataset**

In [2]:
df = pd.read_excel("../data/metadata_df.xlsx")
df.head()

,host_name,host_ip,metric_name,metric_category,metric_value,severity,timestamp,report_date,text,total_hosts,avg_cpu_usage,avg_memory_usage,avg_disk_usage,cpuWarningCount,cpuCriticalCount,memoryWarningCount,memoryCriticalCount,diskWarningCount,diskCriticalCount
0,vbogevokesl02,172.30.5.203,NaN,Memoria,78.29,Advertencia,2024-12-11T10:42:29Z,2024-12-11T15:43:36.334868Z,Host: vbogevokesl02\nIP: 172.30.5.203\nMétrica...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,vbogmosmonp01.mosaico.local,172.30.5.250,NaN,Memoria,75.23,Advertencia,2024-12-11T10:42:28Z,2024-12-11T15:43:36.334868Z,Host: vbogmosmonp01.mosaico.local\nIP: 172.30....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,vbogmosrasp01.mosaico.local,172.30.5.34,NaN,Memoria,84.34,Advertencia,2024-12-11T10:42:31Z,2024-12-11T15:43:36.334868Z,Host: vbogmosrasp01.mosaico.local\nIP: 172.30....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,vbogdtlobsvp09,172.30.5.139,NaN,Memoria,81.81,Advertencia,2024-12-11T10:41:59Z,2024-12-11T15:43:36.334868Z,Host: vbogdtlobsvp09\nIP: 172.30.5.139\nMétric...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,vbogmoststp01.mosaico.local,172.30.5.249,NaN,Memoria,69.98,Normal,2024-12-11T10:41:40Z,2024-12-11T15:43:36.334868Z,Host: vbogmoststp01.mosaico.local\nIP: 172.30....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.columns

Index(['host_name', 'host_ip', 'metric_name', 'metric_category',
       'metric_value', 'severity', 'timestamp', 'report_date', 'text',
       'total_hosts', 'avg_cpu_usage', 'avg_memory_usage', 'avg_disk_usage',
       'cpuWarningCount', 'cpuCriticalCount', 'memoryWarningCount',
       'memoryCriticalCount', 'diskWarningCount', 'diskCriticalCount'],
      dtype='object')

In [4]:
df = df.drop(columns=[ 'total_hosts', 'avg_cpu_usage', 'avg_memory_usage', 'avg_disk_usage', 'cpuWarningCount', 'cpuCriticalCount', 'memoryWarningCount', 'memoryCriticalCount', 'diskWarningCount', 'diskCriticalCount'])

In [5]:
df.head()

,host_name,host_ip,metric_name,metric_category,metric_value,severity,timestamp,report_date,text
0,vbogevokesl02,172.30.5.203,NaN,Memoria,78.29,Advertencia,2024-12-11T10:42:29Z,2024-12-11T15:43:36.334868Z,Host: vbogevokesl02\nIP: 172.30.5.203\nMétrica...
1,vbogmosmonp01.mosaico.local,172.30.5.250,NaN,Memoria,75.23,Advertencia,2024-12-11T10:42:28Z,2024-12-11T15:43:36.334868Z,Host: vbogmosmonp01.mosaico.local\nIP: 172.30....
2,vbogmosrasp01.mosaico.local,172.30.5.34,NaN,Memoria,84.34,Advertencia,2024-12-11T10:42:31Z,2024-12-11T15:43:36.334868Z,Host: vbogmosrasp01.mosaico.local\nIP: 172.30....
3,vbogdtlobsvp09,172.30.5.139,NaN,Memoria,81.81,Advertencia,2024-12-11T10:41:59Z,2024-12-11T15:43:36.334868Z,Host: vbogdtlobsvp09\nIP: 172.30.5.139\nMétric...
4,vbogmoststp01.mosaico.local,172.30.5.249,NaN,Memoria,69.98,Normal,2024-12-11T10:41:40Z,2024-12-11T15:43:36.334868Z,Host: vbogmoststp01.mosaico.local\nIP: 172.30....


# **Load model LLM**

In [31]:
LLM_Llama  = Ollama(model="llama3.2:1b")
LLM_OpenAI = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, max_tokens=None, timeout=None, max_retries=2, api_key=config.api_key)

In [32]:
#agent = create_pandas_dataframe_agent(llm=LLM_OpenAI, df=df, verbose=True, allow_dangerous_code=True)
agent = create_pandas_dataframe_agent(llm=LLM_OpenAI, df=df, agent_type="openai-tools",verbose=True, allow_dangerous_code=True)


# **Example**

In [8]:
agent.invoke("Dame una descripción de la tabla con media y varianza por los 5 host de mayor consumo")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df.groupby('host_name')['metric_value'].agg(['mean', 'var']).nlargest(5, 'mean')"}`


                                  mean         var
host_name                                         
collector                    94.973727    9.943007
vbogmosvbsp01                90.535631  133.874634
vbogevokeml01                89.461600   56.776608
vbogmosansp01.mosaico.local  88.839374    0.030018
vbogmosmonp03.mosaico.local  88.481564    0.508696Los 5 hosts con mayor consumo promedio son:
1. collector con un consumo promedio de 94.97 y una varianza de 9.94
2. vbogmosvbsp01 con un consumo promedio de 90.54 y una varianza de 133.87
3. vbogevokeml01 con un consumo promedio de 89.46 y una varianza de 56.78
4. vbogmosansp01.mosaico.local con un consumo promedio de 88.84 y una varianza de 0.03
5. vbogmosmonp03.mosaico.local con un consumo promedio de 88.48 y una varianza de 0.51

> Finished chain.


{'input': 'Dame una descripción de la tabla con media y varianza por los 5 host de mayor consumo',
 'output': 'Los 5 hosts con mayor consumo promedio son:\n1. collector con un consumo promedio de 94.97 y una varianza de 9.94\n2. vbogmosvbsp01 con un consumo promedio de 90.54 y una varianza de 133.87\n3. vbogevokeml01 con un consumo promedio de 89.46 y una varianza de 56.78\n4. vbogmosansp01.mosaico.local con un consumo promedio de 88.84 y una varianza de 0.03\n5. vbogmosmonp03.mosaico.local con un consumo promedio de 88.48 y una varianza de 0.51'}

# **Prompt**

In [9]:
TEMPLATE = """ Eres un asistente avanzado especializado en el análisis de datos tabulares. Tu misión es ayudar a los compañeros de trabajo a mejorar su experiencia y ayudarles
La tabla actual tiene las siguientes columnas:

- `host_name`: Nombre del host que generó el evento.
- `host_ip`: Dirección IP del host.
- `metric_name`: Nombre de la métrica registrada.
- `metric_category`: Categoría de la métrica.
- `metric_value`: Valor de la métrica registrada.
- `severity`: Severidad del evento (por ejemplo, "Advertencia", "Normal").
- `timestamp`: Marca temporal del evento.
- `report_date`: Fecha y hora en que se generó el reporte.
- `text`: Descripción adicional sobre el evento.

Tus respuestas deben seguir estas reglas, siempre y UNICAMENTE IDIOMA ESPAÑOL:

1. Respuestas textuales simples:
   Si la consulta requiere una respuesta breve coco saludar, lo que no involucra gráficos ni tablas usa este formato:
   {"respuesta": "Tu respuesta aquí en IDIOMA ESPAÑOL"}

   Ejemplo:
   {"respuesta": "La columna 'ventas' tiene el valor promedio más alto."}

2. Si la consulta requiere una tabla, formatee su respuesta usando este formato json:
   {"tabla": {"columna": ["nombre_columna1", "nombre_columna2", ...], "data": [[valor1, valor2, ...], [valor1, valor2, ...], ...]}}
   Ejemplo:
   {"tabla": {"columna": ["host_name", "metric_value", "severity"], "data": [["vbogmosrasp01.mosaico.local", 84.34, "Advertencia"], ["vbogdtlobsvp09", 81.81, "Advertencia"]]}}

3.  Para respuestas que requieren gráficos, responda de esta manera:
   - Para gráficos de barras, usa:
   {"bar": {"columns": ["EjeX1", "EjeX2", ...], "data": [valor1, valor2, ...]}}
   Ejemplo:
   {"bar": {"columna": ["vbogmosrasp01.mosaico.local", "vbogdtlobsvp09"], "data": [84.34, 81.81]}}
   - Para gráficos de líneas, usa:
   {"linea": {"columna": ["EjeX1", "EjeX2", ...], "data": [valor1, valor2, ...]}}
   
4. Errores o falta de datos: Si no puedes responder la consulta por falta de información, usa este formato:
   {"error": "No se encontró suficiente información para responder la consulta."}

**Tu tarea principal es interpretar preguntas y responder UNICAMENTE EN IDIOMA ESPAÑOL de acuerdo con los formatos anteriores, utilizando las columnas y datos de la tabla que te proporcionaré.*** 
"""

In [10]:
model_name = "gpt-3.5-turbo"
encoding = tiktoken.encoding_for_model(model_name)

In [11]:
token_count = len(encoding.encode(TEMPLATE))
print(token_count)

667


# **ChatBot**

In [40]:
question = "Dame una grafica bar con los 5 hosts con mayor consumo de memoria dandome una descripción"

prompt = ChatPromptTemplate.from_messages([
    ("system", TEMPLATE),
    ("human", question)  # Aquí usamos una variable para la entrada del usuario
])

In [41]:
respuesta = agent.invoke({"input": prompt})



> Entering new AgentExecutor chain...
{"bar": {"columns": ["host_name", "metric_value"], "data": [["vbogmosrasp01.mosaico.local", 84.34], ["vbogdtlobsvp09", 81.81], ["vbogevokesl02", 78.29], ["vbogmosmonp01.mosaico.local", 75.23], ["vbogmoststp01.mosaico.local", 69.98]]}}

> Finished chain.


In [42]:
respuesta['output']

'{"bar": {"columns": ["host_name", "metric_value"], "data": [["vbogmosrasp01.mosaico.local", 84.34], ["vbogdtlobsvp09", 81.81], ["vbogevokesl02", 78.29], ["vbogmosmonp01.mosaico.local", 75.23], ["vbogmoststp01.mosaico.local", 69.98]]}}'